In [ ]:
include("benchFunctions.jl")
using PrettyTables
using Printf

In [ ]:
fpf::Int = 32
hex_density::Int = 125

index_density = Dict(125 => 1, 250 => 2, 500 => 3)

if fpf == 32
	println("Using Float32 data")
	example_filepath = "resuts_adjoint_GH200_f32_$(hex_density).csv";
elseif fpf == 64
	println("Using Float64 data")
	example_filepath = "resuts_adjoint_GH200_f64_$(hex_density).csv";
else
	error("fpf must be 32 or 64")
end


my_df = read_special_csv(example_filepath);


# Read specific files
filepaths = ["hex_125.csv", "hex_250.csv", "hex_500.csv"]
data = read_level_csvs(filepaths)

# Combine all data into single DataFrame
combined_df = combine_level_data(data)



for df in values(data)
    df[!, "numbers_mini_tet"] = zeros(Int, nrow(df))
    df[!, "Cumulated_mini_tet"] = zeros(Int, nrow(df))
end

for df in values(data)
    df[!, "numbers_mini_tet"] = df[!, "Level"] .^ 3 .* df[!, "Number_of_elements"]
end

for df in values(data)
    df[!, "Cumulated_mini_tet"] = cumsum(df[!, "numbers_mini_tet"])
end

tot_mini_tets::Vector{Int} = Int[]
for df in values(data)
    push!(tot_mini_tets, sum(df[!, "numbers_mini_tet"]))
    @printf("Total mini tets for %s: %d\n", df[1, "Level"], tot_mini_tets[end])
end

# Display data
println("Data")
for df in values(data)
    println(df)
end



In [ ]:

my_df[!, "Throughput_mini_tet"] = tot_mini_tets[index_density[hex_density]] ./ my_df[!, "clock"];

summary_df = summarize_cluster_stats(my_df);
max_row = col_with_max_Throughput(summary_df);

top = 15
max_row_top = cols_within_max_Throughput(summary_df, top);


println("Top $top rows with maximum Throughput:\n\n");
show(stdout, MIME("text/html"), summary_df);

picture_th = plot_throughput_bars(max_row_top,
    logy=false,
    fpFormat=fpf,
    fig_size=(12, 6),
    fontsize=24,
    plot_title=false);

if fpf == 32
    save_figure_to_pdf(picture_th, "./pdf_plots/throughput_bars_f32.pdf")
elseif fpf == 64
    save_figure_to_pdf(picture_th, "./pdf_plots/throughput_bars_f64.pdf")
end



In [ ]:
summary_df = summarize_cluster_stats(my_df)
println("Cluster stats summary:")
println(summary_df)

In [ ]:
# # Create the 3x1 histogram plot
# hist_fig = plot_level_histograms(data)

# Optional: Create a line plot showing elements vs level
line_fig = plot_elements_by_level(data, fontsize=24, legend_list=["Hex 125", "Hex 250", "Hex 500"])
save_figure_to_pdf(line_fig, "./pdf_plots/elements_by_level.pdf")


In [ ]:
alpha_vec_125 = read_alpha_data("alpha_data_125.dat.gz");
alpha_vec_250 = read_alpha_data("alpha_data_250.dat.gz");
alpha_vec_500 = read_alpha_data("alpha_data_500.dat.gz");

In [ ]:
# println("Alpha Data Statistics:")
# println("=" ^ 50)
# println("Maximum:      $(maximum(alpha_vec))")
# println("Minimum:      $(minimum(alpha_vec))")
# println("Mean:         $(mean(alpha_vec))")
# println("Median:       $(median(alpha_vec))")
# println("Std Dev:      $(std(alpha_vec))")
# println("Count:        $(length(alpha_vec))")
# println("=" ^ 50)

In [ ]:
# # Or with custom parameters
# fig = plot_alpha_histogram(alpha_vec_500, bins=100, color="blue", show_stats=true)
# plt.show()

fig = plot_multiple_alpha_histograms(
    [alpha_vec_125, alpha_vec_250, alpha_vec_500],
    labels=["Alpha 125", "Alpha 250", "Alpha 500"],
    bins=100,
    colors=["blue", "green", "red"],
    show_stats=true,
    figsize=(12, 6),
    cut_at_bin=5.0,
    fontsize=24,
)
plt.show()

if fpf == 32
    save_figure_to_pdf(fig, "./pdf_plots/alpha_hist_f32.pdf")
elseif fpf == 64
    save_figure_to_pdf(fig, "./pdf_plots/alpha_hist_f64.pdf")
end

In [ ]:

# Create the table for your three alpha vectors
alpha_vectors = [alpha_vec_125, alpha_vec_250, alpha_vec_500]
labels = ["Alpha 125", "Alpha 250", "Alpha 500"]

alpha_table = alpha_statistics_table(alpha_vectors, labels);
show(stdout, MIME("text/plain"), alpha_table);


In [ ]:
using Latexify

# Create the table for your three alpha vectors
alpha_vectors = [alpha_vec_125, alpha_vec_250, alpha_vec_500]
labels = ["Alpha 125", "Alpha 250", "Alpha 500"]

alpha_table = alpha_statistics_table(alpha_vectors, labels);

# Convert to LaTeX table with latex=false for plain text
latex_table = latexify(alpha_table, env=:table, fmt="%.6f", latex=false)
println(latex_table)

# Or save to file
open("alpha_statistics.tex", "w") do file
    write(file, latex_table)
end